In [1]:
import os
import datetime
from make_top_box import make_top_protein
from FillWater_AddIons import fill_water_ions
from Energy_Minimization import energy_min
from Nvt_Npt import make_new_minim_nvt_npt
from SAMD import make_new_minim_config_samd
from SAMD import run_md

input_dir = os.getcwd()
    
# pdb file
protein_infile = "HLA_BiAB_protein_50ns"  # 
protein_file_path = os.path.join(input_dir, f"{protein_infile}.pdb")

# MDP files
ions_mdp_file = "ions"
minim_mdp_file = "minim"
nvt_mdp_file = "NVT"
npt_mdp_file = "NPT"
samd_mdp_file = "SAMD"
md_mdp_file = "EngComp_ff14sb_custom"

ions_mdp_path = os.path.join(input_dir, f"{ions_mdp_file}.mdp")
minim_mdp_path = os.path.join(input_dir, f"{minim_mdp_file}.mdp")
nvt_mdp_path = os.path.join(input_dir, f"{nvt_mdp_file}.mdp")
npt_mdp_path = os.path.join(input_dir, f"{npt_mdp_file}.mdp")
samd_mdp_path = os.path.join(input_dir, f"{samd_mdp_file}.mdp")
md_mdp_path = os.path.join(input_dir, f"{md_mdp_file}.mdp")

def create_output_directory():
    # 获取当前目录
    current_dir = os.getcwd()
    
    # 添加时间戳到输出文件夹名称
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir_name = f"output_files_{timestamp}"
    output_dir_path = os.path.join(current_dir, output_dir_name)
    
    # 创建文件夹
    os.mkdir(output_dir_path)
    print(f"Created directory: {output_dir_path}")
    # 切换工作目录到新的输出目录
    os.chdir(output_dir_path)

    
    
    return output_dir_path

In [2]:
create_output_directory()

Created directory: /home/bio/Desktop/jupyter_test/jupyter_test/output_files_20241119_185655


'/home/bio/Desktop/jupyter_test/jupyter_test/output_files_20241119_185655'

In [3]:
# generating a topology and build box
make_top_protein(protein_file_path, "amber99sb-ildn", "tip3p", "system", "topol")

Topology generation completed successfully.
Simulation box definition completed successfully.


In [4]:
# Adding water and ions
fill_water_ions("system", "topol", ions_mdp_path)
# Energy Minimiization
energy_min(minim_mdp_path, "system_ions", "topol", "system_compl")

Water and ions successfully added to the system!
Energy minimization completed successfully!


In [5]:
# Nvt and Npt
sequence = 0
make_new_minim_nvt_npt("system_compl_minim.gro", nvt_mdp_path, npt_mdp_path, "system_equil", 0)

18:57:08 -- Running NVT MD for temperature equilibration...
18:57:27 -- Running NPT MD for pressure equilibration...

Statistics over 50001 steps [ 0.0000 through 50.0000 ps ], 1 data sets
All statistics are over 501 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 293.391        6.2    18.9043    34.8378  (K)

Statistics over 50001 steps [ 0.0000 through 100.0000 ps ], 2 data sets
All statistics are over 501 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                   -2.33981         --    91.0036    12.8364  (bar)
Density                      1004.3         --    1.26851   0.193263  (kg/m^3)
Equilibration completed successfully!


                      :-) GROMACS - gmx energy, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/jupyter_test/output_files_20241119_185655
Command line:
  gmx energy -f NVT.edr -o temp_NVT.xvg

Opened NVT.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Position-Rest.  12  Potential     
 13  Kinetic-En.     14  Total-Energy    15  Conserved-En.   16  Temperature   
 17  Pres.-DC        18  Pressure        19  Constr.-rmsd    20  Vir-XX        
 21  Vir-XY          22  Vir-XZ     

In [6]:
make_new_minim_config_samd("system_equil.gro", samd_mdp_path, "system_Compl_MDstart", 0)

18:57:54 -- Running SAMD 

Statistics over 175001 steps [ 0.0000 through 350.0000 ps ], 3 data sets
All statistics are over 1751 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 339.942         13     27.123   -51.0625  (K)
Pressure                    0.88923         11    97.9889   -47.7903  (bar)
Density                     975.093         13    28.2857    48.7573  (kg/m^3)
Selected 1: 'Protein'
Selected 1: 'Protein'
SAMD completed successfully!


                      :-) GROMACS - gmx energy, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/jupyter_test/output_files_20241119_185655
Command line:
  gmx energy -f SAMD.edr -o press_SAMD.xvg

Opened SAMD.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Box-X           20  Box-Y         
 21  Box-Z           22  Volume 

In [7]:
run_md(md_mdp_path,"system_Compl_MD", "traj_MD", 0, 1)

19:00:02 -- Running MD 


                      :-) GROMACS - gmx energy, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/jupyter_test/output_files_20241119_185655
Command line:
  gmx energy -f PROD.edr -o PROD0.xvg

Opened PROD.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Box-X           20  Box-Y         
 21  Box-Z           22  Volume      


Statistics over 2500001 steps [ 0.0000 through 5000.0000 ps ], 3 data sets
All statistics are over 25001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 309.969      0.034    1.23374 -0.0180104  (K)
Pressure                   -0.27687       0.89    99.4283    2.17067  (bar)
Density                        1006       0.07    2.04417   0.102544  (kg/m^3)


Reading frame     200 time 4000.000   

GROMACS reminds you: "The Poodle Bites" (F. Zappa)



Selected 1: 'Protein'
Selected 1: 'Protein'
Something wrong during Check for MD


NameError: name 'double_check_folder' is not defined

In [11]:
!echo 'Temperature\nPressure\nDensity\n\n' | gmx energy -f SAMD.edr -o press_SAMD.xvg

                      :-) GROMACS - gmx energy, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/jupyter_test/output_files_20241119_181141
Command line:
  gmx energy -f SAMD.edr -o press_SAMD.xvg

Opened SAMD.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Box-X           20  Box-Y         
 21  Box-Z           22  Volume 